In [6]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
from metpy.units import units
from metpy import calc as mpcalc
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, date, timedelta

In [15]:
#Reading in Datasets (odd=hightop, even=lowtop)
ct_ds = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
ct_rgrd_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/ECMWF_ct_rgrd.nc")
t_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/RT_ECMWF_t.nc")
t_2ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/ECCC/RT_ECCC_t.nc")
t_3ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/UKMO/RT_UKMO_t.nc")
t_4ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/CMA/RT_CMA_t.nc")
t_5ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/JMA/RT_JMA_t.nc")
t_6ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/CNR/RT_CNR_t.nc")
rt1_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/realt_jan.nc")
rt2_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/realt_feb.nc")

In [16]:
rt1_ds

<xarray.Dataset>
Dimensions:  (lat: 17, lon: 21, day: 31)
Coordinates:
  * lat      (lat) float64 42.0 40.5 39.0 37.5 36.0 ... 24.0 22.5 21.0 19.5 18.0
    level    int32 ...
  * lon      (lon) float64 198.0 199.5 201.0 202.5 ... 223.5 225.0 226.5 228.0
  * day      (day) int64 1 2 3 4 5 6 7 8 9 10 ... 22 23 24 25 26 27 28 29 30 31
Data variables:
    t        (day, lat, lon) float64 ...

In [9]:
# Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Hour Specification
Hour = 0
hourStr = str(Hour)

#Box coordinates: [[49,-108], [49,-87], [32,-87], [32,-108],[49,-108]]
lat_min = 32
lat_max = 49
lon_min = -108
lon_max = -87

In [10]:
#Regridding ERAI data using xesmf

#ds_out = xr.Dataset(
#    {
#        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
#        "lon": (["lon"], np.arange(0, 360, 1.5)),
#    }
#)

#regridder = xe.Regridder(ct_ds, ds_out, "bilinear")
#regridder

In [11]:
#climot_rgrd = regridder(ct_ds)
#climot_rgrd

In [12]:
#ct_ds = climot_rgrd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ECMWF/ECMWF_ct_rgrd.nc",
#                               mode='w')

In [13]:
#Reducing the datasets
t = t_ds['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
t_rd = t[:,:,:,lat_min:lat_max,lon_min:lon_max]

t2 = t_2ds['t'].sel(level=vlevel)
t2_rd = t2[:,:,:,lat_min:lat_max,lon_min:lon_max]

t3 = t_3ds['t'].sel(level=vlevel)
t3_rd = t3[:,:,:,lat_min:lat_max,lon_min:lon_max]

t4 = t_4ds['t'].sel(level=vlevel)
t4_rd = t4[:,:,:,lat_min:lat_max,lon_min:lon_max]

t5 = t_5ds['t'].sel(level=vlevel)
t5_rd = t5[:,:,:,lat_min:lat_max,lon_min:lon_max]

t6 = t_6ds['t'].sel(level=vlevel)
t6_rd = t6[:,:,:,lat_min:lat_max,lon_min:lon_max]

ct_rgrd_ds = ct_rgrd_ds.reindex(lat=list(reversed(ct_rgrd_ds.lat)))
ct_rd0 = ct_rgrd_ds['t'].sel(lev=vlevel, hour=hourStr,) #<-- this is climo at 1000mb for only 00z
ct_rd = ct_rd0[:,lat_min:lat_max,lon_min:lon_max]

In [14]:
t4_rd

<xarray.DataArray 't' (date: 8, step: 61, number: 3, latitude: 17, longitude: 21)>
[522648 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 198.0 199.5 201.0 202.5 ... 225.0 226.5 228.0
  * latitude   (latitude) float32 42.0 40.5 39.0 37.5 ... 22.5 21.0 19.5 18.0
    level      int32 1000
  * number     (number) int32 1 2 3
  * step       (step) timedelta64[ns] 0 days 1 days 2 days ... 59 days 60 days
  * date       (date) datetime64[ns] 2021-01-04 2021-01-07 ... 2021-01-28
Attributes:
    units:          K
    long_name:      Temperature
    standard_name:  air_temperature

In [ ]:
#Calculating ensemble means
t_ensmn = t_rd.mean(dim='number')
t2_ensmn = t2_rd.mean(dim='number')
t3_ensmn = t3_rd.mean(dim='number')
t4_ensmn = t4_rd.mean(dim='number')
t5_ensmn = t5_rd.mean(dim='number')
t6_ensmn = t6_rd.mean(dim='number')

In [ ]:
#Calculating the area mean temperature
ct_llmn = ct_rd.mean(dim=['lat','lon'])
t_llmn = t_rd.mean(dim=['latitude', 'longitude'])
t2_llmn = t2_rd.mean(dim=['latitude', 'longitude'])
t3_llmn = t3_rd.mean(dim=['latitude', 'longitude'])
t4_llmn = t4_rd.mean(dim=['latitude', 'longitude'])
t5_llmn = t5_rd.mean(dim=['latitude', 'longitude'])
t6_llmn = t6_rd.mean(dim=['latitude', 'longitude'])

In [ ]:
#Writing reduced datasets to NetCDFs
ct_rd_ds = ct_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ct_rd.nc",
                               mode='w')
t_rd_ds = t_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ECMWF_t_rd.nc",
                             mode='w')
t2_rd_ds = t2_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/ECCC_t_rd.nc",
                            mode='w')
t3_rd_ds = t3_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/UKMO_t_rd.nc",
                            mode='w')
t4_rd_ds = t4_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/CMA_t_rd.nc",
                            mode='w')
t5_rd_ds = t5_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/JMA_t_rd.nc",
                            mode='w')
t6_rd_ds = t6_rd.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/rd_t/CNR_t_rd.nc",
                            mode='w')

In [1]:
#Writing Ensemble Mean temps to NetCDFs
t_ensmn_ds = t_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/ECMWF_ensmn_t.nc",
                             mode='w')
t2_ensmn_ds = t2_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/ECCC_ensmn_t.nc",
                            mode='w')
t3_ensmn_ds = t3_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/UKMO_ensmn_t.nc",
                            mode='w')
t4_ensmn_ds = t4_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/CMA_ensmn_t.nc",
                            mode='w')
t5_ensmn_ds = t5_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/JMA_ensmn_t.nc",
                            mode='w')
t6_ensmn_ds = t6_ensmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/ensmn_t/CNR_ensmn_t.nc",
                            mode='w')

NameError: name 't_ensmn' is not defined

In [ ]:
#Writing mean area temps to NetCDFs
ct_llmn_ds = ct_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ct_llmn_t.nc",
                             mode='w')
t_llmn_ds = t_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ECMWF_llmn_t.nc",
                             mode='w')
t2_llmn_ds = t2_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/ECCC_llmn_t.nc",
                            mode='w')
t3_llmn_ds = t3_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/UKMO_llmn_t.nc",
                            mode='w')
t4_llmn_ds = t4_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/CMA_llmn_t.nc",
                            mode='w')
t5_llmn_ds = t5_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/JMA_llmn_t.nc",
                            mode='w')
t6_llmn_ds = t6_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/data/realtime/llmn_t/CNR_llmn_t.nc",
                            mode='w')